In [7]:
import os
import numpy as np
import pandas as pd

import pymatgen as pg


In [8]:
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

In [9]:

def local_min(df):

    df['dy'] = df['Edis'].diff()

    df['sign_dy'] = np.sign(df['dy'])
    df['sign_change'] = df['sign_dy'].diff()

    # Indices of local maxima: derivative goes from negative (-1) to positive (+1)
    local_min_indices = df.index[(df['sign_change'] > 0)]

    local_mins = df.loc[local_min_indices, ['cell_length', 'Edis']].reset_index(drop=True)

    return local_mins

def local_max(df):
    df['dy'] = df['Edis'].diff()

    df['sign_dy'] = np.sign(df['dy'])
    df['sign_change'] = df['sign_dy'].diff()

    # Indices of local maxima: derivative goes from negative (-1) to positive (+1)
    local_min_indices = df.index[(df['sign_change'] < 0)]

    local_max = df.loc[local_min_indices, ['cell_length', 'Edis']].reset_index(drop=True)

    return local_max

In [10]:
def finding_diff(df):
    abs_max = df['Edis'].max()
    max_val = local_max(df)
    min_val = local_min(df)

    diff = (abs_max + max_val['Edis'][0])/2 - min_val['Edis'][0]

    return diff

In [11]:
from pymatgen.core import Structure

In [ ]:
#list all folders inside li_d3s
li_d3s = os.listdir("li_d3s")

df = pd.DataFrame()
cnt=0
for items in li_d3s:
    cnt+=1
    #get the path of each folder
    path = os.path.join("li_d3s", items)
    # print(items)
    
    #ignore .DS_Store
    if items == ".DS_Store":
        continue
    file = os.path.join(path, f'{items}.csv')
    struct_file = os.path.join(path, 'POSCAR')
    df = pd.read_csv(file)
    print(f"Reading {file}")

    struct = Structure.from_file(struct_file)
    sites = struct.num_sites

   
    diff_energy = finding_diff(df)

    print(diff_energy/sites)


print(f'total structure analysed = {cnt}')
    

Reading li_d3s/LiGe_141/LiGe_141.csv
0.0013341615897780837
Reading li_d3s/LiMg_5/LiMg_5.csv
0.016657339849104266
Reading li_d3s/LiB_227/LiB_227.csv
0.0010879295338342499
Reading li_d3s/LiSi_227/LiSi_227.csv
0.001197946015120837
Reading li_d3s/LiCl_225/LiCl_225.csv
0.00696345717606865
Reading li_d3s/LiMn_187/LiMn_187.csv
0.0031490046097577247
Reading li_d3s/LiPd_221/LiPd_221.csv
0.012884955544424075
Reading li_d3s/LiB_62/LiB_62.csv
0.010878998451046082
Reading li_d3s/LiHg_221/LiHg_221.csv
0.006840046432642399
Reading li_d3s/LiMg_164/LiMg_164.csv
0.016591063253382887
Reading li_d3s/LiP_14/LiP_14.csv
0.008016642497761094
Reading li_d3s/LiAl_64/LiAl_64.csv
0.013504645385935585
Reading li_d3s/LiMg_51/LiMg_51.csv
0.003511203079494825
Reading li_d3s/LiSn_10/LiSn_10.csv
0.001558669472609042
Reading li_d3s/LiF_221/LiF_221.csv
0.00812876769215665
Reading li_d3s/LiPb_221/LiPb_221.csv
0.007698734912711974
Reading li_d3s/LiB_194/LiB_194.csv
0.0014141310225483004
Reading li_d3s/LiBr_186/LiBr_186.csv